# Generación datos para LSTM

En este notebook se llevará a cabo la implementación para generar los conjuntos de datos de entrada necesarios para entrenar la red LSTM. La implementación del entrenamiento de la propia red se puede encontrar en el notebook X.

In [1]:
from sklearn.preprocessing import MinMaxScaler
import pickle
import os
import matplotlib.pyplot as plt
from paciente import Paciente
import numpy as np
import warnings

%load_ext autoreload
%autoreload 2

In [2]:
def to_supervised(data, n_input: int, n_out: int, n_margin: int=1, drop_extremes_percentaje:float=0.0):
	"""Convertir los datos a un problema supervisado

	Se transforma el conjunto de datos a un dataset con entradas - salidas.

	Args:
		data (list):  
			Lista con todos los datos de la serie que se quiere transformar.

		n_input (int): 
			Numero de atributos (entradas) de cada muestra a generar.  

		n_out (int): 
			Numero de salias de cada muestra a generar. 

		n_margin (int): 
			Margen entre una muestra y la siguiente que se genera.
		
		drop_extremes_percentaje (float):
			Porcentaje del conjunto de datos que se quiere eliminar de los extremos
			del conjunto de datos. 0.1 indica que se quiere eliminar un 10% del
			inicio de la muestra, y un 10% del final de la muestra.


	Returns:
		list: 
			El primer elemento corresponde con las entradas (X) del conjunto
			de datos y el segundo con las salidas (y).
	"""

	if drop_extremes_percentaje > 0 and drop_extremes_percentaje < 0.5: # Elimino los extremos de la muestra.
		if drop_extremes_percentaje > 0.5:
			print(f"Error. El máximo valor de {drop_extremes_percentaje} es 0.5")
		else:
			data = data[int(len(data)*drop_extremes_percentaje):-int(len(data)*drop_extremes_percentaje)]

	X, y = list(), list()
	in_start = 0

	for _ in range(len(data)):
		# Definimos comienzo y fin de la muestra de entrada y de salida
		in_end = in_start + n_input
		out_end = in_end + n_out
		
		# Comprobamos que tenemos datos suficientes
		if out_end <= len(data):
			x_input = data[in_start:in_end]
			X.append(x_input)
			y.append(data[in_end:out_end])

		# Nos movemos tantos pasos como nos digan en n_margin
		in_start += n_margin
	return np.array(X), np.array(y)

In [3]:
def generar_dataset(smooth_seconds = 5, seconds_prediction = 10, seconds_prior = 30, seconds_margin=1, nombres_ficheros=["data_3.csv", "data_5.csv", "data_7.csv", "data_8.csv"], variables_significativas=['spo2'], drop_extremes_percentaje=0.1):
	"""Función que extrae todos los datos de un conjunto de pacientes

	A partir de la lista de pacientes, y considerando las características del
    conjunto de datos especificadas en los parámetros, se generan los mismos.

	Args:
		smooth_seconds (int):  
			Segundos con los que se aplica la ventana deslizante para realizar el
            suavizado de la señal.

		seconds_prediction (int): 
			Numero de segundos posteriores (salidas) de cada muestra a generar. 

		seconds_prior (int): 
			Numero de segundos previos (entradas) de cada muestra a generar.  

		seconds_margin (int): 
			Margen entre una muestra y la siguiente que se genera.
		
        nombres_ficheros (list):
            Lista de ficheros de pacientes con los que se quiere generar el
            conjunto de datos.
        
        variables_significativas (list):
            Lista de variables significativas que se consideran. Esta lista
            define el número de señales que se usan para generar datos.

		drop_extremes_percentaje (float):
			Porcentaje del conjunto de datos que se quiere eliminar de los extremos
			del conjunto de datos. 0.1 indica que se quiere eliminar un 10% del
			inicio de la muestra, y un 10% del final de la muestra.

	Returns:
		list: 
			Lista de listas con todo el conjunto de datos (cada una de las sublistas
            corresponde con uno de los pacientes).
	"""
    # Indicar la ruta donde se encuentra el fichero que se va a estudiar
    all_data = np.empty(shape=(len(nombres_ficheros), 3), dtype=object)
    dataset_dir = "../data/pacientes"

    X, y = None, None
    all_data = []

    # Leemos cada fichero, y lo añadimos al conjunto de datos
    for i, fichero in enumerate(nombres_ficheros):
        # Indicamos las variables que vamos a querer estudiar. Tienen que ser variables que se encuentren disponibles en el fichero csv
        paciente = Paciente(filename=os.path.join(dataset_dir, fichero), variables_significativas=variables_significativas)
        paciente.comprobar_validez_dataset()

        paciente.suavizar_seniales(smooth_seconds=smooth_seconds)

        # Configuramos el tamaño de entrada y salida de las muestras de la red neuronal
        time_step = paciente.time_step
        n_out = round(seconds_prediction/time_step)

        n_input = round(seconds_prior/time_step)

        n_margin = round(seconds_margin/time_step)

        # Añado al conjunto de datos toda la información de las variables significativas
        data = [paciente.df_smothed[variable_significativa].astype(int) for variable_significativa in paciente.variables_significativas]
        all_data.append(data)

    return all_data

In [4]:
def train_test_val_split(X, y, train_size=0.6, val_size=0.2, test_size=0.2):
    """División en conjuntos de entrenamiento, test y validación

    Args:
        X (list):  
            Lista con los valores de entrada de cada muestra que se desean 
            dividir en conjuntos de entrenamiento.

        y (list):  
            Lista con los valores de salida de cada muestra que se desean 
            dividir en conjuntos de entrenamiento.

        train_size (float): 
            Porcentaje de datos destinados al conjunto de entrenamiento.  

        val_size (float): 
            Porcentaje de datos destinados al conjunto de validacion.  

        test_size (float): 
            Porcentaje de datos destinados al conjunto de test. 

    Returns:
        list: 
            Lista donde cada uno de los elementos corresponde con las 
            siguientes variables (autoexplicativas): X_train, y_train, X_test, 
            y_test, X_val, y_val.
    """

    if train_size + val_size + test_size != 1:
        print("Error, parametros incorrectos")
        return None, None, None

    num_elements_train = int(len(X)* train_size)
    num_elements_test = int(len(X)* test_size)

    return X[:num_elements_train], y[:num_elements_train], X[num_elements_train:num_elements_train+num_elements_test], y[num_elements_train:num_elements_train+num_elements_test], X[num_elements_train+num_elements_test:], y[num_elements_train+num_elements_test:]

## Configuración de variables

En esta sección, se debe indicar la ruta donde se encuentra la información de los pacientes que se desean utilizar para generar el conjunto de datos que utilizará la red LSTM. De igual forma, se deben indicar algunos parámetros para la generación de los datos.

In [5]:
dataset_dir = "../data/pacientes"
lstm_dir = "lstm"
nombres_ficheros = ["data_3.csv", "data_5.csv", "data_7.csv", "data_8.csv"]

seconds_prediction = 10
seconds_prior = 30
seconds_margin=1
time_step = 1/3

n_out = round(seconds_prediction/time_step)
n_input = round(seconds_prior/time_step)
n_margin = round(seconds_margin/time_step)

drop_extremes_percentaje = 0.1

train_size=0.6
val_size=0.2
test_size=0.2

## Generacion dataset univariable

A continuación, se generará el dataset con los parámetros previamente indicados, usando únicamente la señal de saturación como entrada al modelo, y siendo la salida del modelo la señal de saturación.

Cabe destacar que se realiza el escalado de los datos en el rango -1, 1, óptimo para usar la función ReLu como salida de la red LSTM.

In [6]:
all_data = generar_dataset(smooth_seconds=0, seconds_prediction = seconds_prediction, seconds_prior = seconds_prior, seconds_margin = seconds_margin, variables_significativas=["spo2"], nombres_ficheros=nombres_ficheros, drop_extremes_percentaje=drop_extremes_percentaje)

In [7]:
# Extraemos el conjunto de datos que se van a usar para train, para utilizar estos datos para realizar el escalado del dataset
train_data_spo2 = [current_data[:int(len(current_data[0])*train_size)] for current_data in all_data]
train_data_spo2 = np.concatenate(train_data_spo2, axis=1)

In [8]:
# A continuación, escalamos nuestros datos en el rango -1, 1
scaler_spo2 = MinMaxScaler(feature_range=(-1, 1))
scaler_spo2.fit(np.reshape(train_data_spo2, (-1, 1)))
all_data_min_max_spo2 = [scaler_spo2.transform(np.reshape(current_data[0].values, (-1, 1))) for current_data in all_data]

Una vez tenemos todos los datos escalados correctamente, generamos nuestros conjuntos de entrenamiento, validacion y test completos.

In [9]:
x_train = None 
y_train = None
x_test = None
y_test = None
x_val = None
y_val = None

for current_data in all_data_min_max_spo2:
    current_x, current_y = to_supervised(current_data.ravel(), n_input=n_input, n_out=n_out, n_margin=n_margin, drop_extremes_percentaje=drop_extremes_percentaje)

    curr_x_train, curr_y_train, curr_x_test, curr_y_test, curr_x_val, curr_y_val = train_test_val_split(current_x, current_y, train_size=train_size, val_size=val_size, test_size=test_size)

    if x_train is None: x_train = curr_x_train
    else: x_train = np.concatenate((x_train, curr_x_train))

    if y_train is None: y_train = curr_y_train
    else: y_train = np.concatenate((y_train, curr_y_train))

    if x_test is None: x_test = curr_x_test
    else: x_test = np.concatenate((x_test, curr_x_test))
    
    if y_test is None: y_test = curr_y_test
    else: y_test = np.concatenate((y_test, curr_y_test))
        
    if x_val is None: x_val = curr_x_val
    else: x_val = np.concatenate((x_val, curr_x_val))
    
    if y_val is None: y_val = curr_y_val
    else: y_val = np.concatenate((y_val, curr_y_val))

In [10]:
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))
x_val = x_val.reshape((x_val.shape[0], x_val.shape[1], 1))

In [11]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape, x_val.shape, y_val.shape

((49822, 90, 1),
 (49822, 30),
 (16606, 90, 1),
 (16606, 30),
 (16611, 90, 1),
 (16611, 30))

Podemos ver en la celda anterior el tamaño de los conjuntos de entrenamiento, test y validación respectivamente.

In [12]:
with open(f"{lstm_dir}/dataset_univariable.pickle", "wb") as handle:
    pickle.dump([x_train, y_train, x_test, y_test, x_val, y_val, scaler_spo2], handle, protocol=pickle.HIGHEST_PROTOCOL)

## Generacion dataset multivariable

A continuación, se generará el dataset con los parámetros previamente indicados, usando las señales de pulso y saturación como entrada al modelo, y siendo la salida del modelo la señal de saturación.

Cabe destacar que se realiza el escalado de los datos en el rango -1, 1, óptimo para usar la función ReLu como salida de la red LSTM.

In [13]:
all_data = generar_dataset(smooth_seconds=0, seconds_prediction = seconds_prediction, seconds_prior = seconds_prior, seconds_margin = seconds_margin, variables_significativas=["spo2", "pulso"], nombres_ficheros=nombres_ficheros, drop_extremes_percentaje=drop_extremes_percentaje)

In [14]:
# Extraemos el conjunto de datos que se van a usar para train, tanto de la señal del pulso como
# la saturación, para utilizar estos datos para realizar el escalado del dataset
train_data_spo2 = [current_data[0][:int(len(current_data[0])*train_size)] for current_data in all_data]
train_data_spo2 = np.concatenate(train_data_spo2)

train_data_pulso = [current_data[1][:int(len(current_data[1])*train_size)] for current_data in all_data]
train_data_pulso = np.concatenate(train_data_pulso)

In [15]:
# A continuación, escalamos nuestros datos en el rango -1, 1
scaler_spo2 = MinMaxScaler(feature_range=(-1, 1))
scaler_spo2.fit(np.reshape(train_data_spo2, (-1, 1)))
all_data_min_max_spo2 = [scaler_spo2.transform(np.reshape(current_data[0].values, (-1, 1))) for current_data in all_data]

scaler_pulso = MinMaxScaler(feature_range=(-1, 1))
scaler_pulso.fit(np.reshape(train_data_pulso, (-1, 1)))
all_data_min_max_pulso = [scaler_pulso.transform(np.reshape(current_data[1].values, (-1, 1))) for current_data in all_data]

Una vez tenemos todos los datos escalados correctamente, generamos nuestros conjuntos de entrenamiento, validacion y test completos.

In [16]:
x_train = [None, None]
y_train = None
x_test = [None, None]
y_test = None
x_val = [None, None]
y_val = None

for data_spo2_pulso in zip(all_data_min_max_spo2, all_data_min_max_pulso):
    for i, current_data in enumerate(data_spo2_pulso):

        current_x, current_y = to_supervised(current_data.ravel(), n_input=n_input, n_out=n_out, n_margin=n_margin, drop_extremes_percentaje=drop_extremes_percentaje)

        curr_x_train, curr_y_train, curr_x_test, curr_y_test, curr_x_val, curr_y_val = train_test_val_split(current_x, current_y, train_size=0.6, val_size=0.2, test_size=0.2)

        if x_train[i] is None: x_train[i] = curr_x_train
        else: x_train[i] = np.concatenate((x_train[i], curr_x_train))

        if i == 0: # La salida es univariada
            if y_train is None: y_train = curr_y_train
            else: y_train = np.concatenate((y_train, curr_y_train))

        if x_test[i] is None: x_test[i] = curr_x_test
        else: x_test[i] = np.concatenate((x_test[i], curr_x_test))
        
        if i == 0: # La salida es univariada
            if y_test is None: y_test = curr_y_test
            else: y_test = np.concatenate((y_test, curr_y_test))
            
        if x_val[i] is None: x_val[i] = curr_x_val
        else: x_val[i] = np.concatenate((x_val[i], curr_x_val))
        
        if i == 0: # La salida es univariada
            if y_val is None: y_val = curr_y_val
            else: y_val = np.concatenate((y_val, curr_y_val))

In [17]:
x_train = np.stack(x_train, axis=2)
x_test = np.stack(x_test, axis=2)
x_val = np.stack(x_val, axis=2)

In [18]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape, x_val.shape, y_val.shape

((49822, 90, 2),
 (49822, 30),
 (16606, 90, 2),
 (16606, 30),
 (16611, 90, 2),
 (16611, 30))

Podemos ver en la celda anterior el tamaño de los conjuntos de entrenamiento, test y validación respectivamente.

Como podemos observar, la última dimensión de la columna de los datos de entrada al modelo, ha pasado de ser 1 a 2, ya que ahora usamos como entrada no solo la señal de saturación sino también la del pulso

In [19]:
with open(f"{lstm_dir}/dataset_multivariable.pickle", "wb") as handle:
    pickle.dump([x_train, y_train, x_test, y_test, x_val, y_val, scaler_spo2], handle, protocol=pickle.HIGHEST_PROTOCOL)